Needed imports and utility

In [ ]:
import os
import ansys.speos.script as script
import ansys.speos.core as core

tests_data_path = os.path.join(os.path.abspath(''), os.path.pardir, "assets")

def clean_all_dbs(speos_client: core.SpeosClient):
    for item in (
        speos_client.jobs().list()
        + speos_client.scenes().list()
        + speos_client.simulation_templates().list()
        + speos_client.sensor_templates().list()
        + speos_client.source_templates().list()
        + speos_client.intensity_templates().list()
        + speos_client.spectrums().list()
        + speos_client.vop_templates().list()
        + speos_client.sop_templates().list()
        + speos_client.parts().list()
        + speos_client.bodies().list()
        + speos_client.faces().list()
    ):
        item.delete()

# Create connection with speos rpc server

In [ ]:
speos = core.Speos(host="localhost", port=50051)
clean_all_dbs(speos.client)

# Create
## In two steps
### Create locally
The mention "local: " is added when printing the intensity

In [ ]:
intensity1 = script.Intensity(speos_client=speos.client, name="Intensity.1").set_cos(N=1, total_angle=160)
print(intensity1)

### Push it to the server
Now that it is committed to the server, the mention "local: " is no more present when printing the intensity.  
Except for the properties that stays local until a whole project is committed.

In [ ]:
intensity1.commit()
print(intensity1)

## In one step

In [ ]:
intensity2 = script.Intensity(speos_client=speos.client, name="Intensity.2").set_cos(N=2, total_angle=140)\
                                                                            .commit()
print(intensity2)

## Specificity with properties needed
When using a set_xxx, if you observe that there is a set_xxx_properties available, better for you to fill it

In [ ]:
intensity3 = script.Intensity(speos_client=speos.client, name="Intensity.3")
intensity3.set_gaussian().set_FWHM_angle_x(value=40)\
                         .set_FWHM_angle_y(value=30)\
                         .set_total_angle(value=170)\
                         .set_axis_system(axis_system=[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 1.0])
intensity3.commit()
print(intensity3)

## Default values
Some default values are available when applicable in every methods and class.

In [ ]:
intensity4 = script.Intensity(speos_client=speos.client, name="Intensity.4")
print(intensity4)

# Read
## Datamodel
A mention "local: " is added if it is not yet committed to the server

In [ ]:
print(intensity1)

# Update
Tipp: if you are manipulating an intensity already committed, don't forget to commit your changes.  
If you don't, you will still only watch what is committed on the server.

In [ ]:
intensity1.set_cos(N=1, total_angle=50)\
          .commit()
print(intensity1)

## Reset
Possibility to reset local values from the one available in the server.

In [ ]:
intensity3.set_library().set_intensity_file_uri(uri="wrong_file") # set lambertian but no commit
intensity3.reset() # reset -> this will apply the server value to the local value
intensity3.delete() # delete (to display the local value with the below print)
print(intensity3)

# Delete
Once the data is deleted from the server, you can still work with local data and maybe commit later.

In [ ]:
intensity1.delete()
print(intensity1)

In [ ]:
intensity2.delete()
intensity3.delete()

# Cleanup

In [ ]:
clean_all_dbs(speos.client)